# Empirical IO PhD Class
## Problem Set 0
Jimena, Eyal and Pietro 

September 2020

## Problem 0: Logit Function
** 1. The log-sum-exp function is convex everywhere: **

Pick any distinct $x, y \in \mathbb{R}^{N+1}$ and any $\alpha \in (0,1)$. 
$$f ( \alpha x + ( 1 - \alpha ) y ) = \log \sum_{i=0}^N \exp(\alpha x_i + ( 1 - \alpha ) y_i)$$
Applying Hölder's inequality to $\sum_{i=0}^N \exp(\alpha x_i) \exp( ( 1 - \alpha ) y_i)$ with exponents $\frac{1}{\alpha}$ and $\frac{1}{1-\alpha}$ we get 
$$ \sum_{i=0}^N \exp(\alpha x_i) \exp( ( 1 - \alpha ) y_i ) \leq \left [ \sum_{i=0}^N |\exp(\alpha x_i)|^{\frac{1}{\alpha}}\right ]^\alpha \left [ \sum_{i=0}^N |\exp((1 - \alpha) y_i)|^{\frac{1}{1 - \alpha}} \right ]^{1 - \alpha}$$
Taking logs on both sides and rearranging: 
$$ \log \sum_{i=0}^N exp( \alpha x_i + ( 1 - \alpha ) y_i) \leq \alpha \log \sum_{i=0}^N exp( x_i ) + ( 1 - \alpha )\log \sum_{i=0}^N exp( y_i )  $$
So the function is convex everywhere. 

** 2. Using the max trick: **

Fix some $x \in \mathbb{R}^{N+1}$ and let $m:=\max_i x_i$. Assume wlog $x_0=m$. 
We have 
$$ IV = \log \sum_{i=0}^N exp(x_i) = \log \sum_{i=0}^N exp(x_i) \frac{exp(m)}{exp(m)}  $$
and rearranging 
$$ IV = m + \log ( 1 + \sum_{i=1}^N \exp ( x_i-m ) ) $$
We have rescaled everything relative to the $\max$ and added a constant. With this we take the exponential of smaller numbers and avoid the overflow problem. 

** 3. Comparing it to scipy.misc.logsumexp. Does it appear to suffer from underflow/overflow?
Does it use the max trick?**

First generate a tuple of values which includes some $x_i>600$ and evaluate the function at this point.

In [2]:
import numpy as np
import scipy as sp
from scipy.special import logsumexp

x=np.arange(10, 800, 10)


If we calculate the original IV equation we get the error (the evaluated number is infinity):

In [5]:
IV_1=np.log(np.sum(np.exp(x)))
IV_1

c:\users\pietro\anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


inf

Now we do the *max* trick and compare to the value that we get from the logsumexp function:

In [6]:
m=max(x)
IV=m+np.log(1+np.sum(np.exp(x-m)))
IV


790.69316988129776

In [7]:
logsumexp(x)

790.0000454009604

The logsumexp and the function we modified have similar results but not exactly the same so logsumexp is doing something else to compute IV.

## Problem 1
### Stationary Distribution from eigenvectors
Write a function that computes the ergodic distribution of the matrix
$P$ by examining the properly rescaled eigenvectors and compare your result to $P^{100}$.

We first define the transition matrix P and take the 100th power of it.

In [46]:
P = np.array([[0.2, 0.4, 0.4],[0.1, 0.3, 0.6],[0.5, 0.1, 0.4]])
print("This is the transition matrix P \n", P)
P_100 = np.linalg.matrix_power(P, 100)
print("and this is the 100th power \n", P_100)

import quantecon as qe
stat= qe.markov.core.mc_compute_stationary(P)
print("\n this is the stationary using Quantecon package", stat)

This is the transition matrix P 
 [[ 0.2  0.4  0.4]
 [ 0.1  0.3  0.6]
 [ 0.5  0.1  0.4]]
and this is the 100th power 
 [[ 0.31034483  0.24137931  0.44827586]
 [ 0.31034483  0.24137931  0.44827586]
 [ 0.31034483  0.24137931  0.44827586]]

 this is the stationary using Quantecon package [[ 0.31034483  0.24137931  0.44827586]]


The iteration approach and the almighty John Stachurski agree on the stationary distribution. Let us try to compute it using the eigenvector method too.
To calculate the stationary distribution we compute the eigenvalues and left eigenvectors of P. One of the eigenvalues is equal to one as P is stochastic.

In [56]:
P_T = np.matrix.transpose(P)
w,v = np.linalg.eig(P_T)
print("the eigenvalues are \n", w)

the eigenvalues are 
 [ 1.00+0.j         -0.05+0.23979158j -0.05-0.23979158j]


The first eigenvalue is equal to one, so the eigenvector associated to it will be our stationary distribution. Notice that the distribution needs to be a row vector so we need to transpose the eigenvectors we found. Indeed the stationary distribution $\Pi$ satisfies $\Pi' = P'\Pi'$. I.e. it is the transpose of the right eigenvector associated with the unit eigenvalue.

In [64]:
eigenvector = np.real( v.transpose()[0])
print("The eigenvector we consider is \n", eigenvector)

The eigenvector we consider is 
 [-0.52048344 -0.40482045 -0.75180941]


Now we need to rescale it so that the elements sum to one.

In [69]:
normalization = np.sum(eigenvector)
Π = eigenvector / normalization 
print("The ergodic distribution found using eigenvectors is \n",Π)

The ergodic distribution found using eigenvectors is 
 [ 0.31034483  0.24137931  0.44827586]


If we compare with what we found using matrix power, we find the two values are essentially identical.

In [73]:
diff = np.sum ( np.abs(Π -P_100) )
print("The sum of absolute discrepancies between the two matrices is \n",diff)

The sum of absolute discrepancies between the two matrices is 
 1.26842980563e-14


### Stationaty Distribution from system of equations
Just to show off our mathematical prowess, this section solves for the stationary distribution in a different way, solving the system of equations that define it.

We want to have $\pi$ such that $ (P^T-I)\pi=0 $ and $\sum_i \pi_i=1$. We can write this as a system of equations $A\pi=b$ with $A^T=[P^T-I, \mathbb{1}]$ and $b=[\mathbb{0}, 1]^T$ so that we can solve for $\pi$ in $A^T A \pi=A^T b$.

We find the same result as with the other methods.

In [74]:
def stat_distr(M):
    A = np.append(np.matrix.transpose(M) - np.identity(len(M)), [np.ones(len(M))], axis=0)
    A_T = np.matrix.transpose(A)
    b = np.matrix.transpose( np.append( [ np.zeros( len(M)) ], [1] ) )
    x = np.linalg.solve( A_T.dot(A), A_T.dot(b) )
    return x

In [75]:
print("the stationary distribution is \n" , stat_distr(P)) 

the stationary distribution is 
 [ 0.31034483  0.24137931  0.44827586]


## Problem 2: Numerical Integration

We define the function binomial logit using a normal distribution with $\mu=.5$ and $\sigma^2=2$

In [3]:
import scipy as sp
from scipy import stats
def binomiallogit(b, pdf=sp.stats.norm.pdf):
    x=.5
    return (np.exp(b*x)/(1+np.exp(b*x)))*pdf(b, 0.5, np.sqrt(2))
    

    

When we try to integrate over $(-\infty, \infty)$ we get an error, the furthes we can go with the integration limits at around $(-1000, 100)$ so we take this value as the true value of the integral

In [4]:
Quad, err=sp.integrate.quad(binomiallogit, -1000, 1000, epsrel=10**(-14))

In [5]:
mu=.5
sigma=np.sqrt(2)
b1=np.random.normal(mu, sigma, 20)
b2=np.random.normal(mu, sigma, 400)

def fun(t):
    x=.5
    return np.exp(t*x)/(1+np.exp(t*x))

MC20=np.mean([fun(t) for t in b1])
MC400=np.mean([fun(t) for t in b2])

In [6]:
print(MC20)
print(MC400)

0.5143105794690701
0.5623660728833835


In [7]:
def GH(k):
    pts, weigh= np.polynomial.hermite.hermgauss(k)
    return (1/np.sqrt(np.pi))*np.sum(weigh.dot([fun(np.sqrt(2)*sigma*t+mu) for t in pts]))




In [8]:
GH(12)

0.5559391624283003

In [9]:
pts, weigh= np.polynomial.hermite.hermgauss(1)
np.sum(weigh/np.sqrt(np.pi))

1.0

The weights sum up to one. GH with 12 points is the closest to the true value, whereas Monte Carlo with 20 draws is the worst.

In [10]:
print(Quad)
print(MC20)
print(MC400)
print(GH(4))
print(GH(12))

0.555939162843465
0.5143105794690701
0.5623660728833835
0.5559156754781621
0.5559391624283003


Repeat with two dimensions

In [45]:
def binomiallogit2(b1, b2, pdf=sp.stats.multivariate_normal.pdf):
    x=np.array([.5, 1])
    mu=np.array([.5, 1])
    sigma=np.array([[np.sqrt(2), 0], [0,1]])
    return (np.exp(b1*x[0]+b2*x[1])/(1+np.exp(b1*x[0]+b2*x[1])))*pdf([b1, b2], mu, sigma)
    

In [46]:
    
sp.stats.multivariate_normal.pdf(np.array([0, 0]), np.array([.5, 1]), np.array([[np.sqrt(2), 0], [0,1]]))

0.07430684468717029

In [47]:
binomiallogit2(0, 0)

0.037153422343585145

In [49]:
quad2, err2 = sp.integrate.dblquad(binomiallogit2, -1, 1, lambda i: 0, lambda i: 1)

In [51]:
quad2

0.09678615380415492